In [ ]:
data {

    int<lower=1> num_subject;

    int<lower=1> num_trial;

    int<lower=1,upper=2> stimuli[num_subject, num_trial];

    int<lower=1,upper=2> choice[num_subject, num_trial];

    int<lower=0,upper=1> reward[num_subject, num_trial];

}

 

parameters {

    //group level parameters

    real<lower=0,upper=1> group_positive_alpha_mean;

    real<lower=0,upper=0.5> group_positive_alpha_sd;

    real<lower=0,upper=1> group_negative_alpha_mean;

    real<lower=0,upper=0.5> group_negative_alpha_sd;

    real<lower=0,upper=20> group_beta_mean;

    real<lower=0,upper=5> group_beta_sd;

 

    //individual parameters

    real<lower=0,upper=1> positive_alphas[num_subject];

    real<lower=0,upper=1> negative_alphas[num_subject];

    real<lower=0,upper=20> betas[num_subject];

}

 

model {

    vector[2] pchoice;

    real Q[2,2];

    int s; int a;

    real RPE;

    real alpha;

    

    group_positive_alpha_mean ~ normal(0.5,0.1) T[0.1,0.9];

    group_negative_alpha_mean ~ normal(0.5,0.1) T[0.1,0.9];

    group_positive_alpha_sd ~ normal(0.1,0.1) T[0,0.5];

    group_positive_alpha_sd ~ normal(0.1,0.1) T[0,0.5];

    group_beta_mean ~ normal(10,3) T[0,20];

    group_beta_sd ~ normal(2.5,2.5) T[0,5];

 

    for (i in 1:num_subject) {

 

        Q[1,1] = 0.5; Q[1,2] = 0.5; Q[2,1] = 0.5; Q[2,2] = 0.5;

        positive_alphas[i] ~ normal(group_positive_alpha_mean, group_positive_alpha_sd) T[0,1];

        negative_alphas[i] ~ normal(group_negative_alpha_mean, group_negative_alpha_sd) T[0,1];

        betas[i] ~ normal(group_beta_mean, group_beta_sd) T[2,20];

 

        for (j in 1:num_trial) {

            s = stimuli[i,j];

            pchoice[1] = 1/(1+exp(betas[i] * (Q[s,2] - Q[s,1])));

            pchoice[2] = 1-pchoice[1];

            choice[i,j] ~ categorical(pchoice);

            a = choice[i,j];

            RPE = reward[i,j] - Q[s,a];

            alpha = (RPE > 0) ? positive_alphas[i] : negative_alphas[i];

            Q[s,a] = Q[s,a] + alpha * RPE;

        }

    }

}